# Basic implmentation of Clustered Online Cumulative K-Means (CLOCK)

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import cv2

## Importing data

In [2]:
# CNN feature matches
dist = pd.read_csv('./resnet101_matches_distance.csv').values
fname = pd.read_csv('./resnet101_matches_filename.csv').values
# SIFT matches
# good = pd.read_csv('./sift_matches_distance.csv').values
# fgood = pd.read_csv('./sift_matches_filename.csv').values

In [3]:
# Train dataset
train_xy = pd.read_csv('validate_train.csv', index_col=0)
train_path =  pd.read_csv('validate_train.csv')['id'].values
test_path = pd.read_csv('validate.csv')['id'].values
test_xy =  pd.read_csv('validate.csv')

# Running the models

## Main functions

In [4]:
# Limit the candidate pictures to a limited decrease only, comparing to the top one
extract_match = lambda i, thresh: fname[i,np.argwhere(dist[i] < dist[i,0] + thresh)]

## Choosing the biggest cluster for each image, starting from best CNN matches onwards

In [6]:
# Clustered Online Cumulative K-Means (COCK) 
def onl_kmeans(data, fnames, max_clusters, max_range, min_size = 2):
    cluster_centrals = None
    cluster_elems = []
    cluster_filename = []
    cluster_count = []
    for i,coord in enumerate(data):
        # Adding the first point as the first cluster central
        if cluster_centrals is None:
            cluster_centrals = np.array([coord])
            cluster_elems.append([coord])
            cluster_filename.append([fnames[i]])
            cluster_count.append(1)
            continue
        # Get distance from point to each cluster
        distances = np.sum((cluster_centrals - coord)**2, axis=1)**0.5
        nearest = np.argmin(distances)
        # If point is far away from clusters, it's on its own cluster
        if distances[nearest] > max_range:
            # Stop when max number of clusters reached and have a big enough cluster
            if cluster_centrals.shape[0] >= max_clusters:
                if np.max(cluster_count) >= min_size: break
                # Not big enough clusters means that the CNN is messed up
                return None,None,None
            cluster_centrals = np.append(cluster_centrals,[coord], axis=0)
            cluster_elems.append([coord])
            cluster_filename.append([fnames[i]])
            cluster_count.append(1)
        # If not, it belongs to cluster with nearest centeal. Update that one
        else:
            cluster_centrals[nearest] = (cluster_centrals[nearest] 
                                       * cluster_count[nearest] 
                                       + coord) / (cluster_count[nearest]+1)
            cluster_elems[nearest].append(coord)
            cluster_filename[nearest].append(fnames[i])
            cluster_count[nearest] += 1
    # Return the coordinates, filenames, and center of the largest cluster
    biggest_cluster = np.argmax(cluster_count)
    return cluster_elems[biggest_cluster], \
           cluster_filename[biggest_cluster], \
           cluster_centrals[biggest_cluster]

## Running the algorithm, and export the results

In [7]:
# Processing parameters
threshold = 5
max_clusters = 5
max_radius = 8
min_size = 2

locs = []
fnames = []
centroids = []
for i,_ in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    coords = train_xy.loc[img_idx].values
    loc, f, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
    locs.append(loc)
    fnames.append(f)
    centroids.append(centroid)

100%|██████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 1253.27it/s]


In [8]:
# If export cluster image specs for SIFT, run this
f = open(f'./kmeans_coords.pckl','wb')
pickle.dump(locs,f)
f.close()

f = open(f'./fnames_coords.pckl','wb')
pickle.dump(fnames,f)
f.close()

In [13]:
len(test_path)

600

## Partial SIFT Implementation: Only match on large enough clusters

In [19]:
# Processing
threshold = 5
max_clusters = 5
max_radius = 7
min_size = 3

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    coords = train_xy.loc[img_idx].values
    _, _, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
    if centroid is None:
        with open(f'./train_kp/train_kp{test}.pckl', 'rb') as test_sift_file:
            des_test = pickle.load(test_sift_file)
        goods = []

        for train in img_idx:
            with open(f'./train_kp/train_kp{train}.pckl', 'rb') as train_sift_file:
                des_train = pickle.load(train_sift_file)

            # Matching descriptor using KNN algorithm
            if des_train is None or len(des_train) < 2:
                goods.append(-1)
                continue
            matches = flann.knnMatch(des_test,des_train,k=2)

            # Store all good matches as per Lowe's Ratio test.
            good = len([m for m,n in matches if m.distance < 0.7*n.distance])
            goods.append(good)
        
        max_idx = np.argmax(goods)
        centroids.append(train_xy.loc[img_idx[max_idx]].values)
    else:
        centroids.append(centroid)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [03:10<00:00,  3.15it/s]


In [20]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('CLOCK_bigger_cluster.csv',index_label='id',header=['x','y'])

In [21]:
CLOCK_predict = pd.read_csv('CLOCK_bigger_cluster.csv')

# calculate MAE
MAE = np.abs(CLOCK_predict['x']-test_xy['x']) +  np.abs(CLOCK_predict['y']-test_xy['y'])
MAE = np.sum(MAE)/1200
MAE

7.969589224016131

## SIFT implementation: Do feature matching on all CNN candidated pools, choose best matching image

In [23]:
# CLOCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 5
max_radius = 7
min_size = 1

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    
    with open(f'./train_kp/train_kp{test}.pckl', 'rb') as test_sift_file:
        des_test = pickle.load(test_sift_file)
    goods = []

    # Weak finding: Do CLOCK instead
    if des_test is None or len(des_test) < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
        centroids.append(centroid)
        continue

    for train in img_idx:
        with open(f'./train_kp/train_kp{train}.pckl', 'rb') as train_sift_file:
            des_train = pickle.load(train_sift_file)

        # Matching descriptor using KNN algorithm
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)

        # Store all good matches as per Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)
    
    max_idx = np.argmax(goods)
    centroids.append(train_xy.loc[img_idx[max_idx]].values)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [09:36<00:00,  1.04it/s]


In [24]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('CLOCK_CNN_SIFT.csv',index_label='id',header=['x','y'])

In [25]:
CLOCK_predict = pd.read_csv('CLOCK_CNN_SIFT.csv')

# calculate MAE
MAE = np.abs(CLOCK_predict['x']-test_xy['x']) +  np.abs(CLOCK_predict['y']-test_xy['y'])
MAE = np.sum(MAE)/1200
MAE

7.0321777778911105

# Adding histogram matching as a new criteria

In [ ]:
from kmeans import hist_onl_kmeans

## Exhaustive SIFT implmentation: Do feature matching on CNN candidate pool, then do clustering to odd out outliers

In [ ]:
# CLOCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 3
max_radius = 12
min_size = 1
max_match_keep = 0.3

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    # Histogram calculation
    test_img = cv2.imread('./test/' + test + '.jpg')
    t_hist = cv2.calcHist([test_img],[0],None,[256],[0,256])

    img_idx = extract_match(i,threshold).flatten()
    
    sift_fname = fgood[i,:]
    sift_match = good[i,:]

    # Weak finding: Do CLOCK instead
    if sift_fname[0] is np.nan:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, 
                                         threshold, max_radius, min_size)
        centroids.append(centroid)
        continue

    # Get all cnn indices in order of best SIFT matches
    matchings = []
    good_m = []
    for match_idx,m in enumerate(sift_fname):
        if m in img_idx:
            matchings.append(m)
            good_m.append(sift_match[match_idx])
            
    # Once again do thresholding
    good_match = [m for idx,m in enumerate(matchings) 
                  if good_m[idx] > good_m[0]*max_match_keep]

    # Weak finding: Do CLOCK instead
    if good_m[0] < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, 
                                         threshold, max_radius, min_size)
        centroids.append(centroid)
        continue

    coords = train_xy.loc[good_match].values
    _, _, centroid = hist_onl_kmeans(coords, t_hist, good_match, 
                                     max_clusters, max_radius, min_size)
    
    centroids.append(centroid)

100%|██████████| 1200/1200 [01:29<00:00, 13.42it/s]


In [ ]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('CLOCK_CNN_SIFT_cluster.csv',index_label='id',header=['x','y'])

## Exhaustive SIFT, vote between: 
- cluster with best match
- biggest cluster
- cluster with most similar color spectrum

In [ ]:
# CLOCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 5
max_radius = 15
min_size = 1
max_match_keep = 0.4

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    # Histogram calculation
    test_img = cv2.imread('./test/' + test + '.jpg')
    t_hist = cv2.calcHist([test_img],[0],None,[256],[0,256])

    img_idx = extract_match(i,threshold).flatten()
    
    sift_fname = fgood[i,:]
    sift_match = good[i,:]

    # Weak finding: Just get the most similar color
    if sift_fname[0] is np.nan:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, -1, 
                                         max_radius, min_size, 
                                         take_best_hist=True)
        centroids.append(centroid)
        continue

    # Get all cnn indices in order of best SIFT matches
    matchings = []
    good_m = []
    for match_idx,m in enumerate(sift_fname):
        if m in img_idx:
            matchings.append(m)
            good_m.append(sift_match[match_idx])

    # Once again do thresholding
    good_match = [m for idx,m in enumerate(matchings) 
                  if good_m[idx] > good_m[0]*max_match_keep]

    # Weak finding: Just get the most similar color
    if good_m[0] < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, -1, 
                                         max_radius, min_size, 
                                         take_best_hist=True)
        centroids.append(centroid)
        continue

    coords = train_xy.loc[good_match].values
    _, _, centroid = hist_onl_kmeans(coords, t_hist, good_match, max_clusters, 
                                     max_radius, min_size)
    
    centroids.append(centroid)

100%|██████████| 1200/1200 [01:22<00:00, 14.52it/s]


In [ ]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('CLOCK_voting.csv',index_label='id',header=['x','y'])

# Adding SIFT space calculation to try and get a more precise location

In [ ]:
from kmeans import displacement_calculation

# Getting the camera intrinsic matrix
FOV_X = 73.3*np.pi/180
FOV_Y = 53.1*np.pi/180

cx = 680/2
cy = 490/2

fx = cx/np.tan(FOV_X/2)
fy = cy/np.tan(FOV_Y/2)

K = np.array([[fx,0,cx],
              [0,fy,cy],
              [0,0,1]])

In [ ]:
# CLOCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 5
max_radius = 15
min_size = 1
max_match_keep = 0.4
ratio = 0.6
max_range = 10

centroids = []
for i,test in enumerate(tqdm(test_path)):
    test_img = cv2.imread('./test/' + test + '.jpg')
    t_hist = cv2.calcHist([test_img],[0],None,[256],[0,256])

    img_idx = extract_match(i,threshold).flatten()
    
    sift_fname = fgood[i,:]
    sift_match = good[i,:]

    # Weak finding: Just get the most similar color
    if sift_fname[0] is np.nan:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, -1, 
                                         max_radius, min_size, 
                                         take_best_hist=True)
        centroids.append(centroid)
        continue

    # Basically get all cnn indices in order of best SIFT matches
    matchings = []
    good_m = []
    for match_idx,m in enumerate(sift_fname):
        if m in img_idx:
            matchings.append(m)
            good_m.append(sift_match[match_idx])
    # Once again do thresholding
    good_match = [m for idx,m in enumerate(matchings) 
                  if good_m[idx] > good_m[0]*max_match_keep]

    # Weak finding: Just get the most similar color
    if good_m[0] < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, -1, 
                                         max_radius, min_size, 
                                         take_best_hist=True)
        centroids.append(centroid)
        continue

    coords = train_xy.loc[good_match].values
    locs, fnames, centroid = hist_onl_kmeans(coords, t_hist, 
                                             good_match, max_clusters, 
                                             max_radius, min_size)
    final_loc = displacement_calculation(test_img, centroid, locs, 
                                         fnames, K, ratio, max_range)
    centroids.append(final_loc)

100%|██████████| 1200/1200 [10:32<00:00,  1.90it/s]


In [ ]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('SIFT_transform.csv',index_label='id',header=['x','y'])